# Libraries

In [31]:
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import Normalizer
import os
from tensorflow.keras import layers
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Flatten,Embedding,Dense
from keras.layers.merge import Concatenate

import keras

import warnings
warnings.filterwarnings('ignore')



# Preprocessing

In [32]:
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")

test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")


def split_seq(a,split_length):
    x=np.zeros((split_length,a.shape[1]))    
    x=np.vstack([x,a])
    sales_prev=[]
    sales_next=[]

    for i in range(split_length,x.shape[0]): 
        sales_prev.append(x[i-split_length:i])
        sales_next.append(x[i])

    
    return np.array((sales_prev)),np.array((sales_next))

split_len=16    
    
pivoted_train = train.pivot(index=['date'], columns=['store_nbr', 'family'], values=['sales'])

display(pivoted_train)

scaler3 = MinMaxScaler(feature_range=(-1, 1))

pivoted_train = scaler3.fit_transform(pivoted_train)


pivoted_train,pivoted_train_y=split_seq(pivoted_train,split_len)


# Model and Train

In [33]:
def network():
    inp = layers.Input(shape=(split_len, 1782,))
    
    x=keras.layers.LSTM(500,activation="relu",return_sequences=True)(inp)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Dropout(0.3)(x)
    x=keras.layers.LSTM(250,activation="relu")(x)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Dropout(0.3)(x)
    out=keras.layers.Dense(1782)(x)
    
    model = keras.Model(inputs=inp, outputs=out)
    return model

model=network()
print(model.summary())
model.compile(loss=tf.losses.MeanAbsoluteError(),
                optimizer=tf.optimizers.Adam())



model.fit(pivoted_train,pivoted_train_y,batch_size=256,epochs=400,validation_split=0.05)

# Test

In [34]:
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")
id=test['id']

display(test)
test=pivoted_train_y[pivoted_train_y.shape[0]-split_len:]
test=np.expand_dims(test,axis=0)

sales=np.array([])
for i in range(16):
    predictions = model.predict(test)
    predictions=predictions.reshape(-1)
    #sales+=predictions.reshape(-1)
    test=np.insert(test,-1,predictions,axis=1)
    test=np.delete(test, 0, axis=1)
    sales=np.append(sales,predictions)
    #test = np.array([test,predictions])
    #predictions=np.expand_dims(predictions,axis=0)
    #test=np.append(test[1:],predictions,axis=0)


y_predict = pd.DataFrame(scaler3.inverse_transform(sales.reshape((16, 1782))))
sales=y_predict.values.reshape(16*1782)
sales = sales.clip(min=0)
test_results=pd.DataFrame({'id':id,'sales':sales.astype(int)})
display(test_results)
test_results.to_csv('results.csv' , index=False)